In [3]:
import sys
sys.path.insert(0, r'C:\Users\Public\WPy64-39100\notebooks\Anjana Shaji')
import pensionpro_api as pp
from dateutil.relativedelta import relativedelta
from datetime import datetime,timedelta,date
import numpy as np
import time
import pandas as pd
import os
from lxml import etree
from xml.sax.saxutils import escape
start = datetime.now()
print(start)

2024-07-22 15:24:35.211054


In [4]:
d12_df = pd.read_table(r'F:\ASC\USER\All ASC Cases - D12.TXT',index_col=None).astype(object)
d18_df = pd.read_table(r'F:\ASC\USER\All ASC Cases - D18.TXT',index_col=None).astype(object)

d12_df.rename(columns={'S:CLIENTNO': 'tpa_plan_id'}, inplace=True)
d18_df.rename(columns={'S:CLIENTNO': 'tpa_plan_id'}, inplace=True)
output_folder = r'Y:\ASC\Exported Reports\SSAPlanList.txt'
asc_file_folder = r'Y:\ASC\Exported Reports\SSAs'
xml_folder = r'Y:\5500\2022\Automation\8955-SSA'
error_path = r'Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\error_report.xlsx'
os.chdir(asc_file_folder)
xml_df_file = r"Y:\Automation\Team Scripts\Anjana Shaji\8955 SSA DF\df_concat.xlsx"
today_date = date.today()
today_date = datetime.strftime(today_date, "%m-%d-%Y")
print(today_date)

07-22-2024


In [5]:
fake_names = ['ACCOUNT','FORFEITURE','TRANSFER',
              'ZZZ','2020','2021','2022','2023','2024',
              '2025','HOLDING','TAKEOVER','1',' DNU ', 'ZZ', 'DNU', 'UNCASHED', 'Mr. Forfeiture', 'Nan']
ssn_fakes = ['111-11-1111', '222-22-2222', '333-33-3333', '444-44-4444', '555-55-5555', '666-66-6666', '777-77-7777', '888-88-8888', '000-00-0000',
            111-11-1111, 222-22-2222, 333-33-3333, 444-44-4444, 555-55-5555, 666-66-6666, 777-77-7777, 888-88-8888, 000-00-0000]

In [6]:
def find_ein(plan_id, period_start_date):
    sys_plan_id = pp.get_sysplanid(plan_id)
    data = pp.get_plan_by_planid(sys_plan_id)
    plan_number = data['IRSPlanNumber']
    client_id = data['ClientId']
    ein = None
    client_info = pp.get_clients(filters=f"ClientId eq {client_id}", expand="EmployerDatas")
    for client in client_info['Values']:
        plan_cycle = [plan_cycle for plan_cycle in client['EmployerDatas'] if plan_cycle['PeriodStart'] == period_start_date]
        if len(plan_cycle) > 0:
            ein = plan_cycle[0]['EIN']
            print(ein)
    return ein, plan_number

In [7]:
def update_ssa_count(ssa_participant_count, project_id):
    task_group = pp.get_task_groups_by_projectid(project_id, expand = 'tasks.TaskItems')[0]
    for tasks in task_group['Tasks']:
        if tasks['TaskName'] == 'Client corrections':
            for task_item in tasks['TaskItems']:
                if task_item['ShortName'] == 'SSA Participant Count':
                    task_item['Value'] = str(ssa_participant_count)
                    pp.update_taskitem(task_item)

In [8]:
def pad_with_zeros(string):
    string = str(string)
    if len(string) < 4:
        zeros_to_add = 4 - len(string)
        padded_string = '0' * zeros_to_add + string
        return padded_string
    else:
        return string

In [9]:
def check_if_asc_file_exists(plan_id, period_end_date, asc_file_list):
    found_flag = False
    file_name = None
    for file in asc_file_list:
        file_plan_id = file.split("_")[0]
        file_period_end = file.split("_")[1]
        file_period_end_dt = datetime.strptime(file_period_end,'%m%d%Y')
        file_period_end_dt = file_period_end_dt + timedelta(hours=12)
        if file_plan_id == plan_id and file_period_end_dt == period_end_date:
            found_flag = True
            file_name = file
    return found_flag, file_name 
    

In [10]:
def get_asc_files():
    asc_files = os.listdir(asc_file_folder)
    asc_file_list = [file for file in asc_files if 'FormSSA' in file and file.endswith('.txt') and not file.startswith('_')]
    return asc_file_list

In [11]:
def get_ssa_and_asc_df(df):
    file_found_index = []
    plan_id_dict = {}
    df_error_log = pd.read_excel(error_path)
    if df.empty:
        raise SystemExit("Script is shutting down")
    for index, row in df.iterrows():
        plan_id = row['planid']
        period_end_date = row['per_end']
        period_end_date = datetime.strptime(period_end_date,'%m/%d/%Y %H:%M:%S %p')
        asc_file_list = get_asc_files()
        file_exists_flag, file_name = check_if_asc_file_exists(plan_id, period_end_date, asc_file_list)
        if file_exists_flag:
            print('ASC file found for plan: ', plan_id)
            file_found_index.append(index)
            plan_id_dict[plan_id] = file_name
        else:
            today = date.today()
            today = datetime.strftime(today, "%Y-%m-%d") 
            projects = pp.get_projects_by_planid(plan_id,filters=f"Name eq 'Form 8955-SSA (Automated)'")
            project = [project for project in projects if project['CompletedOn'] is None]
            if len(project) > 0:
                project = project[0]
            project_id = project['Id']
            task_group = pp.get_task_groups_by_projectid(project_id, expand = 'Tasks.Taskitems')[0]
            for task in task_group['Tasks']:
                if task['TaskName'] == 'Completion - Form 8955-SSA':
                    task_id = task['Id']
                    for task_item in task['TaskItems']:
                        updated = task_item['UpdatedOn'] 
                        updated = updated.rsplit('T')[0]
                        if task_item['ShortName'] == '8955 Validation Error' and task_item['Value'] == None:
                            task_item['Value'] = 'Complete'
                            pp.update_taskitem(task_item)
                            note_text = 'No ASC file found'
                            payload = {
                                "ProjectID": project_id, 
                                "NoteText": f"{note_text}",
                                "NoteCategoryId": 3514,
                                "ShowOnPSL": False
                                    }
                
                            x = pp.add_note(payload)
                        elif task_item['ShortName'] == '8955 Validation Error' and task_item['Value'] == 'Complete' and not updated == today:
                            pp.override_task(task_id)
            df_error_log.loc[df_error_log['TPA Plan ID'].astype(str).str.contains(plan_id), 'ASC Text File Missing'] = 'Yes'
    df_error_log.to_excel(error_path, index = False)
    df_ssa_target = df.loc[file_found_index]
    df_ssa_target.reset_index(drop=True, inplace=True)
    return df_ssa_target, plan_id_dict

In [12]:
df = pp.get_worktray('Automation', get_all=True)
filt1 = df['task_name'] == 'Completion - Form 8955-SSA'
filt2 = df['proj_name'] == 'Form 8955-SSA (Automated)'
df = df[filt1 & filt2]
print(len(df))

100


In [13]:
df

,projid,taskid,task_name,task_active,daystocomp,planid,plan_name,proj_name,plan_status,plan_category,...,form5500,plan_end,effective_on,pbgc,proj_due_on,per_start,per_end,plan_group,admin_type,priority
840,10220347,72240934,Completion - Form 8955-SSA,1/30/2024 12:00:00 AM,4,10457,"Dana Manshadi DDS, PLLC 401(k) P/S Plan",Form 8955-SSA (Automated),Pending Termination,NOT ON PDM - Client Elected Out of PDM Service.,...,5500 SF,1/31,1/1/2019 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608
849,10317231,72804488,Completion - Form 8955-SSA,2/23/2024 12:00:00 AM,4,16202,Trasca 401(k) Plan,Form 8955-SSA (Automated),Terminated,PDM - Yes,...,5500 SF,2/28,1/1/2021 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608
1011,10841855,75558794,Completion - Form 8955-SSA,7/9/2024 12:00:00 AM,1,18265,Central Automotive Group 401(k) Plan,Form 8955-SSA (Automated),Active,PDM - Yes,...,Large Plan Filer,12/31,1/1/2023 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608
1017,10841910,75559228,Completion - Form 8955-SSA,7/9/2024 12:00:00 AM,1,4658,Saint Arnold Brewing Company 401(k) Profit Sha...,Form 8955-SSA (Automated),Active,PDM - Yes,...,Large Plan Filer,12/31,1/1/2023 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608
1018,10841911,75559243,Completion - Form 8955-SSA,7/9/2024 12:00:00 AM,1,91846,Patients Medical Center 401(k) Plan,Form 8955-SSA (Automated),Active,PBA PDM,...,Large Plan Filer,12/31,1/1/2018 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2264,10902511,75860736,Completion - Form 8955-SSA,7/19/2024 12:00:00 AM,1,6247,Axis Pipe and Tube 401(k) Plan,Form 8955-SSA (Automated),Fired - Pending Final Services,PDM - Yes,...,Large Plan Filer,12/31,1/1/2018 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608
2266,10904120,75878771,Completion - Form 8955-SSA,7/20/2024 12:00:00 AM,1,11879,Buda Dental Professionals 401(k) Plan,Form 8955-SSA (Automated),Active,PDM - Yes,...,5500 SF,12/31,1/1/2020 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608
2267,10904126,75878819,Completion - Form 8955-SSA,7/20/2024 12:00:00 AM,1,91870,"WTKJP, Inc. 401(k) Plan",Form 8955-SSA (Automated),Pending Termination,PBA PDM,...,5500 SF,12/31,1/1/2019 12:00:00 AM,N/A,5/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,7/31/2023 12:00:00 AM,N/A,Full Administration,21608
2268,10904128,75878835,Completion - Form 8955-SSA,7/20/2024 12:00:00 AM,1,6100,The Council on Recovery 403(b) Plan,Form 8955-SSA (Automated),Active,PDM - Yes,...,Large Plan Filer,12/31,1/1/2018 12:00:00 AM,N/A,10/15/2024 12:00:00 AM,1/1/2023 12:00:00 AM,12/31/2023 12:00:00 AM,N/A,Full Administration,21608


In [14]:
df_ssa_target, plan_id_dict = get_ssa_and_asc_df(df)
len(df_ssa_target)

ASC file found for plan:  18265
ASC file found for plan:  3517
ASC file found for plan:  10457
ASC file found for plan:  19426
ASC file found for plan:  90697
ASC file found for plan:  18666
ASC file found for plan:  16368
ASC file found for plan:  12161
ASC file found for plan:  17690
ASC file found for plan:  16559
ASC file found for plan:  15394
ASC file found for plan:  19489
ASC file found for plan:  16801
ASC file found for plan:  12263
ASC file found for plan:  12605
ASC file found for plan:  5502
ASC file found for plan:  9880
ASC file found for plan:  90567
ASC file found for plan:  10292
ASC file found for plan:  16462
ASC file found for plan:  10229
ASC file found for plan:  15894
ASC file found for plan:  15918
ASC file found for plan:  19259
ASC file found for plan:  17618
ASC file found for plan:  10997
ASC file found for plan:  19118
ASC file found for plan:  11028
ASC file found for plan:  17647
ASC file found for plan:  16408
ASC file found for plan:  9163
ASC file fou

76

In [15]:
len(df_ssa_target)

76

In [16]:
plan_name_list1 = df['planid'].tolist()
plan_name_list2 = df_ssa_target['planid'].tolist()
for plan in plan_name_list1:
    if plan not in plan_name_list2:
        print(plan)


16202
4658
91846
4869
2566
4537
10808
3327
3686
5908
8380
12157
5366
9611
10216
10145
11228
10336
17949
12402
11890
91870
11059


In [18]:
# Find the ASC file for each plan and concatenate all the data into one dataframe df_concat
cols = ['EIN','plannumber','year','form','entrycode','SSN','firstname','initial','lastname','annuitycode','payfreqcode','b','tot','b2','b3']
df_concat = pd.DataFrame(columns=cols)
df_error_log = pd.read_excel(error_path)
for index, row in df_ssa_target.iterrows():
    plan_id =  row['planid']
    print(plan_id)
    period_end_date = row['per_end']
    period_start_date = row['per_start']
    project_id = row['projid']
    plan_name = row['plan_name']
    year = period_end_date.rsplit(' ')[0].rsplit('/')[2]
    if year == '2024':
        print('Year is 2024')
        continue
    ssa_participant_count = None
    close_out_flag = False
    file_name = plan_id_dict[plan_id]
    try:
        df_asc_file = pd.read_table(file_name, skiprows=1, header=None, names=cols, dtype=str, encoding='ansi')
    except Exception as e:
        print(e)
        df_asc_file = pd.DataFrame()
    if df_asc_file.empty:
        print('Empty')
        close_out_flag = True
    else:
        df_counts = df_asc_file[df_asc_file['entrycode'] == "Counts"]
        last_count_indices = df_counts.index
        if len(last_count_indices) > 0:
            last_count_index = last_count_indices[-1]
            last_count_index_adjusted = last_count_index + 2
            df_final = pd.read_table(file_name, skiprows=last_count_index_adjusted, header=None, names=cols, dtype=str, encoding='ansi')
            
            if df_final.empty:
                print('Empty')
                close_out_flag = True
            else:
                df_final = df_final[df_final['year'] == year]
                try:
                    ein, plan_number = find_ein(plan_id, period_start_date)
                    df_final['EIN'] = ein
                    df_final['plannumber'] = plan_number
                except Exception as e:
                    print(e)
                    print('Error finding EIN in PRO')
                df_final.loc[df_final['SSN'].astype(str).str.startswith('9'), 'SSN'] = 'FOREIGN'
                df_final.loc[df_final['SSN'].astype(str).str.startswith('000') | df_final['SSN'].astype(str).str.startswith('666'), 'SSN'] = 'FOREIGN'
                
                df_final.loc[df_final['SSN'].astype(str).str.contains('-00-'), 'SSN'] = 'FOREIGN'
                df_final.loc[df_final['SSN'].astype(str).str.contains('-0000'), 'SSN'] = 'FOREIGN'
                ssn_mask = df_final['SSN'].isin(ssn_fakes)
                df_final.loc[ssn_mask, 'SSN'] = '999-99-9999'
                contains_letters = df_final['SSN'].astype(str).str.contains(r'[a-zA-Z]', na=False)
                df_final.loc[contains_letters, 'SSN'] = 'FOREIGN'
                
                firstname_mask = df_final['firstname'].isin(fake_names)
                lastname_mask = df_final['lastname'].isin(fake_names)
                initial_mask = df_final['initial'].isin(fake_names)
                df_final.loc[firstname_mask | lastname_mask | initial_mask, 'SSN'] = '999-99-9999'
                df_final.loc[df_final['firstname'].isna() | (df_final['firstname'] == ''), 'SSN'] = '999-99-9999'
                df_final.loc[df_final['lastname'].isna() | (df_final['lastname'] == ''), 'SSN'] = '999-99-9999'
                
                
                df_final['tot2'] = df_final.tot.fillna(0).astype(float).apply(lambda x: f'{x:.0f}')
                df_final.loc[(df_final['entrycode'] == 'A') & (df_final['tot2'].astype(int) < 1) & (df_final['tot2'].astype(int) >= 0), 'tot2'] = '1'
                df_final.tot2.replace('0', pd.NaT, inplace=True)
                df_final = df_final[df_final['SSN'] != '999-99-9999']
                if df_final.empty:
                    print('Empty')
                    close_out_flag = True
                else:
                    df_final['planid'] = plan_id
                    df_final['projid'] = project_id
                    df_final['A'] = len(df_final[df_final.entrycode == 'A'])
                    df_final['A'] = df_final['A'].apply(lambda x: f'{x:.0f}')
                    df_final['D'] = len(df_final[df_final.entrycode == 'D']) 
                    df_final['planname'] = plan_name

                    df_concat = pd.concat([df_concat, df_final], ignore_index=True)
                    try:
                        a_sum = df_final['A'].astype(int).iloc[0]
                        d_sum = df_final['D'].astype(int).iloc[0]
                        ssa_participant_count = a_sum + d_sum 
                        update_ssa_count(ssa_participant_count, project_id)
                    except Exception as e:
                        continue
        elif len(last_count_indices) == 0:
            df_final = pd.read_table(file_name, skiprows=1, header=None, names=cols, dtype=str, encoding='ansi')
            if df_final.empty:
                print('Empty')
                close_out_flag = True
            else:
                df_final = df_final[df_final['year'] == year]
                try:
                    ein, plan_number = find_ein(plan_id, period_start_date)
                    df_final['EIN'] = ein
                    df_final['plannumber'] = plan_number
                except Exception as e:
                    print(e)
                    print('Error finding EIN in PRO')
                df_final.loc[df_final['SSN'].astype(str).str.startswith('9'), 'SSN'] = 'FOREIGN'
                df_final.loc[df_final['SSN'].astype(str).str.startswith('000') | df_final['SSN'].astype(str).str.startswith('666'), 'SSN'] = 'FOREIGN'
                df_final.loc[df_final['SSN'].astype(str).str.contains('-00-'), 'SSN'] = 'FOREIGN'
                df_final.loc[df_final['SSN'].astype(str).str.contains('-0000'), 'SSN'] = 'FOREIGN'
                ssn_mask = df_final['SSN'].isin(ssn_fakes)
                df_final.loc[ssn_mask, 'SSN'] = '999-99-9999'
                contains_letters = df_final['SSN'].astype(str).str.contains(r'[a-zA-Z]', na=False)
                df_final.loc[contains_letters, 'SSN'] = 'FOREIGN'
                firstname_mask = df_final['firstname'].isin(fake_names)
                lastname_mask = df_final['lastname'].isin(fake_names)
                initial_mask = df_final['initial'].isin(fake_names)
                df_final.loc[firstname_mask | lastname_mask | initial_mask, 'SSN'] = '999-99-9999'
                df_final.loc[df_final['firstname'].isna() | (df_final['firstname'] == ''), 'SSN'] = '999-99-9999'
                df_final.loc[df_final['lastname'].isna() | (df_final['lastname'] == ''), 'SSN'] = '999-99-9999'
                df_final['tot2'] = df_final.tot.fillna(0).astype(float).apply(lambda x: f'{x:.0f}')
                df_final.loc[(df_final['entrycode'] == 'A') & (df_final['tot2'].astype(int) < 1) & (df_final['tot2'].astype(int) >= 0), 'tot2'] = '1'
                df_final.tot2.replace('0', pd.NaT, inplace=True)
                df_final = df_final[df_final['SSN'] != '999-99-9999']
                if df_final.empty:
                    print('Empty')
                    close_out_flag = True
                else:
                    df_final['planid'] = plan_id
                    df_final['projid'] = project_id
                    df_final['A'] = len(df_final[df_final.entrycode == 'A'])
                    df_final['A'] = df_final['A'].apply(lambda x: f'{x:.0f}')
                    df_final['D'] = len(df_final[df_final.entrycode == 'D']) 
                    df_final['planname'] = plan_name
                    df_concat = pd.concat([df_concat, df_final], ignore_index=True)
                    try:
                        a_sum = df_final['A'].astype(int).iloc[0]
                        d_sum = df_final['D'].astype(int).iloc[0]
                        ssa_participant_count = a_sum + d_sum
                        update_ssa_count(ssa_participant_count, project_id)
                    except Exception as e:
                        continue
    if  close_out_flag:
        print('The project needs to be closed out for plan: ', plan_id)
        note_text = 'No participants to report, project unnecessary.'
        payload = {
                "ProjectID": project_id, 
                "NoteText": f"{note_text}",
                "NoteCategoryId": 3251,
                "ShowOnPSL": False
                    }

        x = pp.add_note(payload)
        pp.close_project(project_id)
        df_error_log.loc[df_error_log['TPA Plan ID'].astype(str).str.contains(plan_id), 'Project Closed Out'] = 'Yes'
        
df_error_log.to_excel(error_path, index = False)     
df_concat    

18265
Empty
The project needs to be closed out for plan:  18265
3517
Year is 2024
10457
Year is 2024
19426
330670046
Empty DataFrame
Columns: [EIN, plannumber, year, form, entrycode, SSN, firstname, initial, lastname, annuitycode, payfreqcode, b, tot, b2, b3, tot2]
Index: []
Empty
The project needs to be closed out for plan:  19426
90697
Year is 2024
18666
Year is 2024
16368
Year is 2024
12161
Year is 2024
17690
Year is 2024
16559
Year is 2024
15394
Year is 2024
19489
522229904
         EIN plannumber  year form entrycode          SSN  firstname initial  \
0  522229904        003  2023  SSA         A  230-49-1055   KIMBERLY     NaN   
1  522229904        003  2023  SSA         A  225-68-6362     YVONNE     NaN   
2  522229904        003  2023  SSA         A  170-62-1123  FRANCISCO     NaN   
3  522229904        003  2023  SSA         A  199-78-9619      KILEY     NaN   

   lastname annuitycode payfreqcode    b        tot   b2   b3    tot2  
0  CASANOVA           A           A  NaN    

742037298
         EIN plannumber  year form entrycode          SSN firstname initial  \
0  742037298        002  2023  SSA         D  440-72-0722   ANTHONY     NaN   

  lastname annuitycode payfreqcode    b  tot   b2   b3 tot2  
0   CHAVEZ         NaN         NaN  NaN  NaN  NaN  NaN  NaT  
12034
550915682
Empty DataFrame
Columns: [EIN, plannumber, year, form, entrycode, SSN, firstname, initial, lastname, annuitycode, payfreqcode, b, tot, b2, b3, tot2]
Index: []
Empty
The project needs to be closed out for plan:  12034
3679
750738122
         EIN plannumber  year form entrycode          SSN firstname initial  \
0  750738122        001  2023  SSA         D  454-35-2281   CESARIO     NaN   

  lastname annuitycode payfreqcode    b  tot   b2   b3 tot2  
0     MATA         NaN         NaN  NaN  NaN  NaN  NaN  NaT  
16700
Year is 2024
5664
300544256
         EIN plannumber  year form entrycode          SSN firstname initial  \
0  300544256        001  2023  SSA         D  403-47-3625      

475423814
Empty DataFrame
Columns: [EIN, plannumber, year, form, entrycode, SSN, firstname, initial, lastname, annuitycode, payfreqcode, b, tot, b2, b3, tot2]
Index: []
Empty
The project needs to be closed out for plan:  18102
15999
932051256
         EIN plannumber  year form entrycode          SSN firstname initial  \
0  932051256        001  2023  SSA         A  095-92-3285    DANIEL     NaN   

    lastname annuitycode payfreqcode    b      tot   b2   b3  tot2  
0  CERVANTES           A           A  NaN  1012.25  NaN  NaN  1012  
4920
203697647
Empty DataFrame
Columns: [EIN, plannumber, year, form, entrycode, SSN, firstname, initial, lastname, annuitycode, payfreqcode, b, tot, b2, b3, tot2]
Index: []
Empty
The project needs to be closed out for plan:  4920
2534
810953387
Empty DataFrame
Columns: [EIN, plannumber, year, form, entrycode, SSN, firstname, initial, lastname, annuitycode, payfreqcode, b, tot, b2, b3, tot2]
Index: []
Empty
The project needs to be closed out for plan:  253

KeyboardInterrupt: 

In [16]:
df_concat.to_excel(xml_df_file)

In [17]:
# Generate the xml file that needs to be uploaded to DGEM 	
str_header = '<root Version="1.0">'
str_footer = '</root>'
str_partinfo = ''
xmlstring = ''
str3 = '</Form>'
for planid in df_concat.planid.drop_duplicates()[:]:   
    df1 = df_concat[df_concat['planid'] == planid].astype(str).fillna(value='').copy()
    # str_clientinfo = '<Form><Form5558ApplicationFiledInd>1</Form5558ApplicationFiledInd><EIN>' + df1['EIN'].iloc[0] + '</EIN><PlanName>' + escape(df1['planname'].iloc[0]) + '</PlanName><PlanNumber>' + df1['plannumber'].iloc[0] + '</PlanNumber><!-- Line 6a --><ReqPartCnt>' + df1['A'].iloc[0] + '</ReqPartCnt><!-- Line 6b --><VolPartCnt>0</VolPartCnt><!-- Line 7  --><TotalPartCnt>' + df1['A'].iloc[0] + '</TotalPartCnt>'        
    str_clientinfo = '<Form><EIN>' + df1['EIN'].iloc[0] + '</EIN><PlanName>' + escape(df1['planname'].iloc[0]) + '</PlanName><PlanNumber>' + df1['plannumber'].iloc[0] + '</PlanNumber><!-- Line 6a --><ReqPartCnt>' + df1['A'].iloc[0] + '</ReqPartCnt><!-- Line 6b --><VolPartCnt>0</VolPartCnt><!-- Line 7  --><TotalPartCnt>' + df1['A'].iloc[0] + '</TotalPartCnt>'        
    partstring = ''
    print(df1['planname'].iloc[0])
    for i in df1.index:
        if df1['entrycode'].loc[i] == 'A':
            if df1['initial'].loc[i] == 'nan':
                partstring = '<Participant><EntryCode>A</EntryCode><SSN>' + df1['SSN'].loc[i] + '</SSN><FirstName>' + escape(df1['firstname'].loc[i]) + '</FirstName><LastName>' + escape(df1['lastname'].loc[i]) + '</LastName><AnnuityCode>A</AnnuityCode><PayFreqCode>A</PayFreqCode><DcTotValAccAmt>' + df1['tot2'].loc[i] + '</DcTotValAccAmt></Participant>'
            else:
                partstring = '<Participant><EntryCode>A</EntryCode><SSN>' + df1['SSN'].loc[i] + '</SSN><FirstName>' + escape(df1['firstname'].loc[i]) + '</FirstName><Initial>' + escape(df1['initial'].loc[i]) + '</Initial><LastName>' + escape(df1['lastname'].loc[i]) + '</LastName><AnnuityCode>A</AnnuityCode><PayFreqCode>A</PayFreqCode><DcTotValAccAmt>' + df1['tot2'].loc[i] + '</DcTotValAccAmt></Participant>'
        elif df1['entrycode'].loc[i] == 'B':
            if df1['initial'].loc[i] == 'nan':
                partstring = '<Participant><EntryCode>B</EntryCode><SSN>' + df1['SSN'].loc[i] + '</SSN><FirstName>' + escape(df1['firstname'].loc[i]) + '</FirstName><LastName>' + escape(df1['lastname'].loc[i]) + '</LastName><AnnuityCode>A</AnnuityCode><PayFreqCode>A</PayFreqCode><DcTotValAccAmt>' + df1['tot2'].loc[i] + '</DcTotValAccAmt></Participant>'
            else:
                partstring = '<Participant><EntryCode>B</EntryCode><SSN>' + df1['SSN'].loc[i] + '</SSN><FirstName>' + escape(df1['firstname'].loc[i]) + '</FirstName><Initial>' + escape(df1['initial'].loc[i]) + '</Initial><LastName>' + escape(df1['lastname'].loc[i]) + '</LastName><AnnuityCode>A</AnnuityCode><PayFreqCode>A</PayFreqCode><DcTotValAccAmt>' + df1['tot2'].loc[i] + '</DcTotValAccAmt></Participant>'
        else:
            if df1['initial'].loc[i] == 'nan':
                partstring = '<Participant><EntryCode>D</EntryCode><SSN>' + df1['SSN'].loc[i] + '</SSN><FirstName>' + escape(df1['firstname'].loc[i]) + '</FirstName><LastName>' + escape(df1['lastname'].loc[i]) + '</LastName></Participant>'
            else:
                partstring = '<Participant><EntryCode>D</EntryCode><SSN>' + df1['SSN'].loc[i] + '</SSN><FirstName>' + escape(df1['firstname'].loc[i]) + '</FirstName><Initial>' + escape(df1['initial'].loc[i]) + '</Initial><LastName>' + escape(df1['lastname'].loc[i]) + '</LastName></Participant>'
        str_partinfo += partstring
        partstring = ''
    xmlstring += str_clientinfo + str_partinfo + str3
    str_clientinfo = ''
    str_partinfo = ''
    print('done!')
        
finalxmlstring = str_header + xmlstring + str_footer
root = etree.fromstring(finalxmlstring)
tree = etree.ElementTree(root)
print('write xml')
tree.write(f'{xml_folder}/{today_date} Export 2.xml', pretty_print=True, xml_declaration=True, encoding="utf-8")
print('XML file is ready!')

Automated Solutions, LLC 401(k) Plan - 002
done!
Bonded Logic, Inc. 401(k) Profit Sharing Plan
done!
Avondale Group 401(k) Plan
done!
DFDG Architecture 401(k) Plan
done!
BetterWorld Tech Inc. 401(k) and Profit Sharing Plan
done!
SEI 401(k) Plan
done!
ISTS 401(k) Plan
done!
Davis and Sons Doors, LLC 401(k) Profit Sharing Plan
done!
Beaumont Manufacturing & Distribution Co. Retirement Plan
done!
Beyond Engineering and Testing 401(k) Plan
done!
Bob Moore Construction, Inc. 401(k) Profit Sharing Plan
done!
Caliper Energy Services 401(K) and Retirement Plan
done!
Carolina Pediatric Dentistry 401(k) Profit Sharing Plan
done!
Cenla RV 401(k) Plan
done!
Charleston Radiologists, PA 401(k) Profit Sharing Plan
done!
ChaseSource 401(k) Savings and Profit Sharing Plan
done!
Citrus Dental of Inverness, P.A. 401(k) Plan
done!
Cleveland Lumber Company 401(k) Plan
done!
Clouse Brown 401(k) Plan
done!
Coastal RV 401(k) Profit Sharing Plan
done!
Concord Eye Care Center 401(k) Profit Sharing Plan
done!
Co

El Paso Trade School, Inc. Western Technical College 401(k) Plan
done!
Douglass W. King Co., Inc. Retirement Plan
done!
Dula Construction 401(k) Plan
done!
P97 Networks 401(k) Plan
done!
B & H Construction 401(k) Plan
done!
Civility Partners, Inc. 401(k) Plan
done!
Envirochem, Inc. 401(k) Profit Sharing Plan
done!
Three Crosses Regional Hospital, LLC 401(k) Plan
done!
4h Ohana, LLC 401(k) Plan
done!
Action Onsite, Inc. 401(k) Plan
done!
LKN Capital Mortgage 401(k) Plan
done!
Restoration Professionals, Inc. 401(k) Profit Sharing Plan
done!
Contech Control Services 401(k) Plan
done!
Diality, Inc. 401(k) Plan
done!
The Angell Law Firm 401(k) Plan
done!
Bay Oil Company Retirement Plan
done!
Christy Buck Real Estate Team Retirement Plan
done!
Holiday Management, Inc. 401(k) Plan
done!
Mueller Water Conditioning, Inc. 401(k) Plan
done!
Fishburne Pharmacy LLC 401(k) Profit Sharing Plan
done!
TPS Gulf Southeast, LLC. 401(k) Plan
done!
Jefferson Gulf Coast Management LLC Retirement Plan
done!
A

In [18]:
end = datetime.now()
diff = end - start 
minutes = diff.total_seconds() / 60
print('minutes: ',minutes)

minutes:  72.64453048333333
